In [1]:
import scrapy, os
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.exporters import CsvItemExporter
from scrapy.linkextractors import LinkExtractor
from scrapy.item import DictItem, Field

In [2]:
def create_item_class(class_name,field_list):
    field_dict = {}
    for field_name in field_list:
        field_dict[field_name] = Field()

    return type(class_name, (DictItem,), {'fields':field_dict})


In [3]:
class Pipeline(object):
    """Scrapy Pipeline Object for Links, Results, Team, Individual"""
    
    def __init__(self):
        self.files = {}
        self.exporters = {}
    
    def create_file(self, file, _dir): 
        # if the file doesn't exist create it 
        if file not in self.files:
            self.files[file] = open(os.path.join(_dir, file + '.csv'), 'wb')
            self.exporters[file] = CsvItemExporter(self.files[file])
            self.exporters[file].start_exporting()
    
    def close_spider(self, spider):
        [e.finish_exporting() for e in self.exporters.values()]
        [f.close() for f in self.files.values()]
 
    def process_item(self, item, spider):
        # Define what item type it is 
        file = type(item).__name__.replace("item_", "").lower()
        if "link" in file: 
            _dir = "Links" 
        else: 
            _dir = "Data"
        
        self.create_file(file, _dir)
        
        # Export 
        self.exporters[file].export_item(item)
        
        return item

In [4]:
class StatsSpider(scrapy.Spider): 
    name = "GamebyGame"
    regex = 'http\:\/\/stats\.ncaa\.org\/player\/index\?id\=\d+\&org_id\=\d+\&stats_player_seq\=\-\d+\&year_stat_category_id\=\d+'
    
    def start_requests(self): 
        file_in = 'gamebygamelinks.csv'

        # Pull in the data 
        data = pd.read_csv(os.path.join("Links", file_in))
        _list = data.to_dict(orient='record')
        
        # Loop through the links 
        for url in _list[:1]: 
            print(url['link'])
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse,
                                 meta = {'team':url['team'], 
                                         'year' :url['year'],
                                        }
                                )
    
    def parse(self, response): 
            
        # Get all the links on the page
        le = LinkExtractor(allow = self.regex) 
        links = le.extract_links(response)
        
        # Build up the output 
        for link in links: 
            stat = link.text.replace(" ", "").replace("/", "").replace(".", "")
            yield scrapy.Request(url = link.url, 
                                 callback = self.parse_data, 
                                 meta = {'team': response.meta['team'], 
                                         'year': response.meta['year'],
                                         'stat': stat
                                        }
                                )
            
            
    def parse_data(self, response): 
        
        # Create tables 
        tables = pd.read_html(response.body)
        
        # Create yearly table 
        yearly = self.yearlystats(tables[2])
        
        # Create dynamic items
        stat = response.meta['stat'].replace(" ","").replace("/","").replace(".","").replace("-", "")
        field_list = yearly.columns
        DynamicItem = create_item_class("yearly_" + stat, field_list)
        item = DynamicItem()
        
        # yield items
        for record in yearly.to_dict(orient='record'):
            for k, v in record.items(): 
                item[k] = v
            yield item
        
        # Create game-by-game table 
        gamebygame = self.gamestats(tables[4], response.meta['team'])
        
        # Create dynamic items
        field_list = gamebygame.columns
        DynamicItem = create_item_class('gamebygame_' + stat, field_list)
        item = DynamicItem()
        
        # yield items
        for record in gamebygame.to_dict(orient='record'):
            for k, v in record.items(): 
                item[k] = v
            yield item
        
        
    def yearlystats(self, table):
        """Clean up the yearly game-to-game stats table
            table : pandas DF you want to clean 
        """
        # Drop unneeded header row
        tmp = table.iloc[1:,]
        # Create a header row removing spaces, /'s, and x's
        tmp.columns = [x.replace(" ", "").replace("/","").replace(".","") for x in tmp.iloc[0]]
        # Drop the row used to create header row
        tmp = tmp.drop(tmp.index[0])
        # Forward fill the year for analysis later 
        tmp['Year'].fillna(method='ffill', inplace = True)
        # Create a new offense/defense variable 
        tmp['OffenseDefense'] = tmp['Team']
        # Figure out which team we are working with 
        curr_team = tmp.iloc[:1,1:3].values[0][0]
        # Create a new team variable
        tmp['Team'] = curr_team
        # In the offense defense variable, fill in the offense defense variable 
        tmp['OffenseDefense'] = tmp['OffenseDefense'].apply(lambda x: 'Offense' if x == curr_team else x)
        return tmp 
    
    def gamestats(self, table, curr_team):
        """ Clean up game stats table
            table : pandas DF you want to clean 
            curr_team : team name you want to add 
        """

        # Drop unneeded header 
        tmp = table.iloc[1:,]
        # Fix the column names by reading line 0
        tmp.columns = [x.replace(" ", "").replace("/","").replace(".","") for x in tmp.iloc[0]]
        # Drop row zero which held the header row
        tmp = tmp.drop(tmp.index[0])
        # Forward fill the dates for defensive split later 
        tmp['Date'].fillna(method='ffill', inplace = True)
        # Add in the team 
        tmp['Team'] = curr_team
        # Create an offense/defense variable
        tmp['OffenseDefense'] = tmp['Opponent']
        # If it's not a defensive total then it's offense - set that in the offensedefense variable
        tmp['OffenseDefense'] = tmp['OffenseDefense'].apply(lambda x: "Defense" if x == "Defensive Totals" else "Offense")
        # Set the defensive totals in the opponent varaible to nullls
        tmp['Opponent'] = tmp['Opponent'].apply(lambda x: None if x == "Defensive Totals" else x)
        # Forward fill the opponents in for analysis later
        tmp['Opponent'].fillna(method='ffill', inplace = True)
        # Forward fill the results in for analysis later 
        tmp['Result'].fillna(method='ffill', inplace = True)
        return tmp
        
    


In [5]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                          'ITEM_PIPELINES': {'__main__.Pipeline':100}
                         })

process.crawl(StatsSpider)
process.start()

2017-12-11 14:56:44 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-11 14:56:44 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-11 14:56:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-11 14:56:44 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

http://stats.ncaa.org/player/game_by_game?game_sport_year_ctl_id=11520&org_id=721&stats_player_seq=-100


2017-12-11 14:56:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/player/game_by_game?game_sport_year_ctl_id=11520&org_id=721&stats_player_seq=-100> (referer: None)
2017-12-11 14:56:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10420> (referer: http://stats.ncaa.org/player/game_by_game?game_sport_year_ctl_id=11520&org_id=721&stats_player_seq=-100)
2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10420>
{'Blkd': '7',
 'FumblesRecovered': '5',
 'G': '12',
 'GP': '12',
 'Int': '4',
 'OffenseDefense': 'Offense',
 'Team': 'Air Force',
 'Year': '2013-14'}
2017-12-11 14:56:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10440> (referer: http://stats.ncaa.

2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10420>
{'Blkd': nan,
 'Date': '10/05/2013',
 'FumblesRecovered': '1',
 'G': '1/',
 'Int': '2',
 'OffenseDefense': 'Defense',
 'Opponent': '@ Navy',
 'Result': 'L 10 - 28',
 'Team': 'Air Force Falcons'}
2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10420>
{'Blkd': '1',
 'Date': '10/10/2013',
 'FumblesRecovered': '1',
 'G': '1/',
 'Int': '1',
 'OffenseDefense': 'Offense',
 'Opponent': 'San Diego St.',
 'Result': 'L 20 - 27',
 'Team': 'Air Force Falcons'}
2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10420>
{'Blkd': nan,
 'Date': '10/10/2013',
 'FumblesRecovered': '1',
 'G': '1/',
 'Int': n

2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10440>
{'FumblesLost': '8',
 'FumblesRecovered': '9',
 'GP': '0',
 'Int': '8',
 'Interceptions': '15',
 'OffenseDefense': 'Defensive',
 'Team': 'Air Force',
 'Year': '2016-17'}
2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10440>
{'FumblesLost': '14',
 'FumblesRecovered': '5',
 'GP': '12',
 'Int': '5',
 'Interceptions': '6',
 'OffenseDefense': 'Offense',
 'Team': 'Air Force',
 'Year': '2017-18'}
2017-12-11 14:56:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10440>
{'FumblesLost': '5',
 'FumblesRecovered': '14',
 'GP': '0',
 'Int': '6',
 'Interceptions': '5',
 'OffenseDefense': 'Defensive',
 'Team': 'Air 

...

2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'FirstDownsbyPenalty': '15',
 'G': '13',
 'GP': '13',
 'OffenseDefense': 'Offense',
 'PassYards': '1893',
 'PassingFirstDowns': '74',
 'Penalties': '46',
 'PenaltiesPerGame': '3.54',
 'PenaltyYards': '460',
 'PenaltyYdsPerGame': '35.38',
 'Plays': '1002',
 'RushNetYards': '3550',
 'RushingFirstDowns': '198',
 'TOP': '24607',
 'Team': 'Air Force',
 'TotOff': '5443',
 'TotalOffYardsG': '418.69',
 'YdsPlay': '0.00',
 'Year': '2014-15'}
2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'FirstDownsbyPenalty': '14',
 'G': '13',
 'GP': '0',
 'OffenseDefense': 'Defensive',
 'PassYards': '3369',
 'PassingFirstDowns': '144',
 'Penalties': '59',
 'PenaltiesPerGame': '4.54',
 'PenaltyYards': '535',
 'Pena

2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'Date': '09/21/2013',
 'FirstDownsbyPenalty': '1',
 'G': '1/',
 'OffenseDefense': 'Offense',
 'Opponent': 'Wyoming',
 'PassYards': '127',
 'PassingFirstDowns': '7',
 'Penalties': '7',
 'PenaltiesPerGame': '7.00',
 'PenaltyYards': '61',
 'PenaltyYdsPerGame': '61.00',
 'Plays': '77',
 'Result': 'L 23 - 56',
 'RushNetYards': '346',
 'RushingFirstDowns': '16',
 'TOP': '1889',
 'Team': 'Air Force Falcons',
 'TotOff': '473',
 'TotalOffYardsG': '473.00',
 'YdsPlay': nan}
2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'Date': '09/21/2013',
 'FirstDownsbyPenalty': '4',
 'G': '1/',
 'OffenseDefense': 'Defense',
 'Opponent': 'Wyoming',
 'PassYards': '373',
 'PassingFirstDowns': '19',
 'Penalties': '10

2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'Date': '11/08/2013',
 'FirstDownsbyPenalty': nan,
 'G': '1/',
 'OffenseDefense': 'Defense',
 'Opponent': '@ New Mexico',
 'PassYards': '28',
 'PassingFirstDowns': '1',
 'Penalties': '2',
 'PenaltiesPerGame': '2.00',
 'PenaltyYards': '15',
 'PenaltyYdsPerGame': '15.00',
 'Plays': '68',
 'Result': 'L 37 - 45',
 'RushNetYards': '451',
 'RushingFirstDowns': '26/',
 'TOP': '2104',
 'Team': 'Air Force Falcons',
 'TotOff': '479',
 'TotalOffYardsG': '479.00',
 'YdsPlay': nan}
2017-12-11 14:57:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10394>
{'Date': '11/21/2013',
 'FirstDownsbyPenalty': nan,
 'G': '1/',
 'OffenseDefense': 'Offense',
 'Opponent': 'UNLV',
 'PassYards': '70',
 'PassingFirstDowns': '3',
 'Penalties': '3'

2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10393>
{'Date': '09/07/2013',
 'G': '1/',
 'OffenseDefense': 'Offense',
 'Opponent': 'Utah St.',
 'Rec': '6',
 'RecTD': '1',
 'RecYdsG': nan,
 'ReceivingYards': '108',
 'ReceptionsPerGame': nan,
 'Result': 'L 20 - 52',
 'Team': 'Air Force Falcons',
 'YardsPerReception': nan}
2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10393>
{'Date': '09/07/2013',
 'G': '1/',
 'OffenseDefense': 'Defense',
 'Opponent': 'Utah St.',
 'Rec': '32',
 'RecTD': '5',
 'RecYdsG': nan,
 'ReceivingYards': '360',
 'ReceptionsPerGame': nan,
 'Result': 'L 20 - 52',
 'Team': 'Air Force Falcons',
 'YardsPerReception': nan}
2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_

2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10393>
{'Date': '11/21/2013',
 'G': '1/',
 'OffenseDefense': 'Defense',
 'Opponent': 'UNLV',
 'Rec': '15',
 'RecTD': nan,
 'RecYdsG': '141.00',
 'ReceivingYards': '141',
 'ReceptionsPerGame': '15.00',
 'Result': 'L 21 - 41',
 'Team': 'Air Force Falcons',
 'YardsPerReception': '9.40'}
2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/player/index?id=11520&org_id=721&stats_player_seq=-100&year_stat_category_id=10393>
{'Date': '11/30/2013',
 'G': '1/',
 'OffenseDefense': 'Offense',
 'Opponent': '@ Colorado St.',
 'Rec': '11',
 'RecTD': nan,
 'RecYdsG': '125.00',
 'ReceivingYards': '125',
 'ReceptionsPerGame': '11.00',
 'Result': 'L 13 - 58',
 'Team': 'Air Force Falcons',
 'YardsPerReception': '11.36'}
2017-12-11 14:57:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.or